# Advanced K-Means Clustering

## Learning Objectives
- Understand K-Means algorithm in depth
- Learn how to determine optimal number of clusters
- Implement Elbow Method and Silhouette Analysis
- Apply K-Means to real-world datasets
- Visualize clustering results effectively

## What is K-Means?
K-Means is an unsupervised learning algorithm that groups similar data points into K clusters.

### How it works:
1. Initialize K centroids randomly
2. Assign each point to nearest centroid
3. Update centroids as mean of assigned points
4. Repeat steps 2-3 until convergence

### Key Parameters:
- `n_clusters`: Number of clusters to form
- `init`: Method for initialization ('k-means++' recommended)
- `n_init`: Number of times algorithm runs with different centroid seeds
- `max_iter`: Maximum number of iterations
- `random_state`: For reproducibility

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

## 1️⃣ Basic K-Means on Synthetic Data
Let's start with a simple example using synthetic data to understand the algorithm.

In [ ]:
# Create synthetic dataset with clear clusters
X, y_true = make_blobs(
    n_samples=500,
    centers=4,
    cluster_std=0.8,
    random_state=42
)

print(f"Dataset shape: {X.shape}")
print(f"Number of true clusters: {len(np.unique(y_true))}")

# Visualize the data
plt.figure(figsize=(10, 6))
plt.scatter(X[:, 0], X[:, 1], c=y_true, cmap='viridis', alpha=0.6, edgecolors='k')
plt.title('Synthetic Dataset with True Labels', fontsize=14, fontweight='bold')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.colorbar(label='True Cluster')
plt.show()

In [ ]:
# Apply K-Means clustering
kmeans = KMeans(
    n_clusters=4,
    init='k-means++',  # Smart initialization
    n_init=10,         # Run 10 times with different initializations
    max_iter=300,
    random_state=42
)

# Fit and predict
y_pred = kmeans.fit_predict(X)

# Get cluster centers
centers = kmeans.cluster_centers_

# Visualize results
plt.figure(figsize=(10, 6))
plt.scatter(X[:, 0], X[:, 1], c=y_pred, cmap='viridis', alpha=0.6, edgecolors='k')
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=300, alpha=0.8, 
            marker='X', edgecolors='black', linewidths=2, label='Centroids')
plt.title('K-Means Clustering Results', fontsize=14, fontweight='bold')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.colorbar(label='Predicted Cluster')
plt.show()

print(f"\nInertia (Sum of squared distances): {kmeans.inertia_:.2f}")
print(f"Number of iterations: {kmeans.n_iter_}")

## 2️⃣ Determining Optimal Number of Clusters

### Elbow Method
The Elbow Method helps us find the optimal K by plotting inertia vs number of clusters.
We look for the "elbow" point where adding more clusters doesn't significantly reduce inertia.

In [ ]:
# Calculate inertia for different values of K
inertias = []
K_range = range(2, 11)

for k in K_range:
    kmeans_temp = KMeans(n_clusters=k, init='k-means++', n_init=10, random_state=42)
    kmeans_temp.fit(X)
    inertias.append(kmeans_temp.inertia_)

# Plot Elbow Curve
plt.figure(figsize=(10, 6))
plt.plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Number of Clusters (K)', fontsize=12)
plt.ylabel('Inertia (Within-Cluster Sum of Squares)', fontsize=12)
plt.title('Elbow Method for Optimal K', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xticks(K_range)

# Highlight the elbow point (K=4 in this case)
plt.axvline(x=4, color='r', linestyle='--', linewidth=2, label='Optimal K=4')
plt.legend()
plt.show()

### Silhouette Analysis
Silhouette Score measures how similar a point is to its own cluster compared to other clusters.
- Range: [-1, 1]
- Higher values indicate better-defined clusters
- Values near 0 indicate overlapping clusters
- Negative values indicate misclassified points

In [ ]:
# Calculate silhouette scores for different K values
silhouette_scores = []

for k in K_range:
    kmeans_temp = KMeans(n_clusters=k, init='k-means++', n_init=10, random_state=42)
    labels = kmeans_temp.fit_predict(X)
    score = silhouette_score(X, labels)
    silhouette_scores.append(score)
    print(f"K={k}: Silhouette Score = {score:.3f}")

# Plot Silhouette Scores
plt.figure(figsize=(10, 6))
plt.plot(K_range, silhouette_scores, 'go-', linewidth=2, markersize=8)
plt.xlabel('Number of Clusters (K)', fontsize=12)
plt.ylabel('Silhouette Score', fontsize=12)
plt.title('Silhouette Analysis for Optimal K', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xticks(K_range)

# Highlight the best K
best_k = K_range[np.argmax(silhouette_scores)]
plt.axvline(x=best_k, color='r', linestyle='--', linewidth=2, 
            label=f'Best K={best_k}')
plt.legend()
plt.show()

## 3️⃣ Real-World Application: Customer Segmentation
Let's apply K-Means to segment customers based on their purchasing behavior.

In [ ]:
# Load diamonds dataset
df = sns.load_dataset('diamonds')
print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Select numerical features for clustering
features = ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']
X_diamonds = df[features].copy()

# Check for missing values
print("Missing values:")
print(X_diamonds.isnull().sum())

# Remove any rows with zero dimensions (data quality issue)
X_diamonds = X_diamonds[(X_diamonds['x'] > 0) & (X_diamonds['y'] > 0) & (X_diamonds['z'] > 0)]
print(f"\nDataset shape after cleaning: {X_diamonds.shape}")

# Feature scaling is CRUCIAL for K-Means!
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_diamonds)

print("\nFeatures scaled successfully!")

In [ ]:
# Find optimal K for diamonds dataset
# Note: Using a sample for computational efficiency
sample_size = 5000
X_sample = X_scaled[np.random.choice(X_scaled.shape[0], sample_size, replace=False)]

inertias_diamonds = []
silhouette_diamonds = []
K_range_diamonds = range(2, 9)

for k in K_range_diamonds:
    kmeans_temp = KMeans(n_clusters=k, init='k-means++', n_init=10, random_state=42)
    labels = kmeans_temp.fit_predict(X_sample)
    inertias_diamonds.append(kmeans_temp.inertia_)
    silhouette_diamonds.append(silhouette_score(X_sample, labels))

# Plot both metrics
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Elbow plot
ax1.plot(K_range_diamonds, inertias_diamonds, 'bo-', linewidth=2, markersize=8)
ax1.set_xlabel('Number of Clusters (K)', fontsize=12)
ax1.set_ylabel('Inertia', fontsize=12)
ax1.set_title('Elbow Method - Diamonds Dataset', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Silhouette plot
ax2.plot(K_range_diamonds, silhouette_diamonds, 'go-', linewidth=2, markersize=8)
ax2.set_xlabel('Number of Clusters (K)', fontsize=12)
ax2.set_ylabel('Silhouette Score', fontsize=12)
ax2.set_title('Silhouette Analysis - Diamonds Dataset', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print scores
for k, sil in zip(K_range_diamonds, silhouette_diamonds):
    print(f"K={k}: Silhouette Score = {sil:.3f}")

In [ ]:
# Apply K-Means with optimal K (let's use K=3 based on analysis)
optimal_k = 3
kmeans_final = KMeans(n_clusters=optimal_k, init='k-means++', n_init=10, random_state=42)
df['cluster'] = kmeans_final.fit_predict(X_scaled)

print(f"\nCluster distribution:")
print(df['cluster'].value_counts().sort_index())

# Analyze cluster characteristics
print("\nCluster Characteristics (Mean Values):")
cluster_summary = df.groupby('cluster')[features].mean()
print(cluster_summary)

In [ ]:
# Visualize clusters using PCA for dimensionality reduction
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(12, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df['cluster'], 
                     cmap='viridis', alpha=0.6, edgecolors='k', s=50)
plt.xlabel(f'First Principal Component ({pca.explained_variance_ratio_[0]:.2%} variance)', fontsize=12)
plt.ylabel(f'Second Principal Component ({pca.explained_variance_ratio_[1]:.2%} variance)', fontsize=12)
plt.title('Diamond Clusters Visualized with PCA', fontsize=14, fontweight='bold')
plt.colorbar(scatter, label='Cluster')
plt.grid(True, alpha=0.3)
plt.show()

print(f"\nTotal variance explained by 2 components: {pca.explained_variance_ratio_.sum():.2%}")

## 4️⃣ Business Insights
Let's interpret the clusters and derive actionable insights.

In [ ]:
# Create comprehensive cluster profiles
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Price distribution by cluster
df.boxplot(column='price', by='cluster', ax=axes[0, 0])
axes[0, 0].set_title('Price Distribution by Cluster')
axes[0, 0].set_xlabel('Cluster')
axes[0, 0].set_ylabel('Price ($)')

# Carat distribution by cluster
df.boxplot(column='carat', by='cluster', ax=axes[0, 1])
axes[0, 1].set_title('Carat Distribution by Cluster')
axes[0, 1].set_xlabel('Cluster')
axes[0, 1].set_ylabel('Carat')

# Cluster sizes
cluster_counts = df['cluster'].value_counts().sort_index()
axes[1, 0].bar(cluster_counts.index, cluster_counts.values, color='skyblue', edgecolor='black')
axes[1, 0].set_title('Cluster Sizes')
axes[1, 0].set_xlabel('Cluster')
axes[1, 0].set_ylabel('Number of Diamonds')

# Average price by cluster
avg_price = df.groupby('cluster')['price'].mean()
axes[1, 1].bar(avg_price.index, avg_price.values, color='lightcoral', edgecolor='black')
axes[1, 1].set_title('Average Price by Cluster')
axes[1, 1].set_xlabel('Cluster')
axes[1, 1].set_ylabel('Average Price ($)')

plt.suptitle('Comprehensive Cluster Analysis', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 📊 Key Takeaways

### Algorithm Understanding:
1. K-Means is sensitive to initialization (use k-means++)
2. Feature scaling is essential
3. Works best with spherical clusters

### Determining Optimal K:
1. **Elbow Method**: Look for the "elbow" in the inertia plot
2. **Silhouette Score**: Higher is better (closer to 1)
3. **Domain Knowledge**: Consider business context

### Best Practices:
1. Always scale features before clustering
2. Try multiple values of K
3. Use multiple evaluation metrics
4. Visualize results when possible
5. Interpret clusters in business context

### Limitations:
1. Assumes spherical clusters
2. Sensitive to outliers
3. Requires specifying K beforehand
4. May converge to local optima

### When to Use K-Means:
- ✅ Customer segmentation
- ✅ Image compression
- ✅ Document clustering
- ✅ Anomaly detection (preprocessing)
- ❌ Non-spherical clusters (use DBSCAN)
- ❌ Varying cluster densities (use hierarchical clustering)

## 🎯 Practice Exercises

1. **Try different datasets**: Apply K-Means to Iris or Wine datasets
2. **Feature engineering**: Create new features and see how they affect clustering
3. **Compare algorithms**: Try DBSCAN or Hierarchical Clustering on the same data
4. **Outlier handling**: Remove outliers and observe the impact
5. **Mini-batch K-Means**: For large datasets, try MiniBatchKMeans for faster computation